In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import jsonlines
import itertools


In [2]:
NER = ["ISOLATE"]

In [3]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [4]:
df = df.reset_index().drop(columns=["index"])

In [5]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[We, also, found, 20, nonribosomally, synthesi...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.956822395324707, -3.364196300506592, 7.22...","[-100, 2, 2, 2, 2, 2, -100, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 4, 4, 4, 5, 6, 7, 8, 9, ...",ISOLATE
1,"[The, isobutanol, biosynthesis, pathway, in, S...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.948723077774048, -3.370908737182617, 7.29...","[-100, 2, 2, -100, -100, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 1, 1, 2, 3, 4, 5, 5, 6, 6, 6, 7, ...",ISOLATE
2,"[Strain, ME23, specifically, was, identified, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",2,"[[-2.8552932739257812, -3.23327374458313, 6.88...","[-100, 2, 2, -100, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11...",ISOLATE
3,"[BRCA2, tumours, expressed, cyclin, D1, and, D...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.735277652740478, -3.456892967224121, 7.22...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",ISOLATE
4,"[One, contained, 12, of, the, 14, milk, strain...","[O, O, O, O, O, O, B, O, O, O, O, O, O, O, O, ...",4,"[[-2.889482736587524, -3.250913143157959, 7.06...","[-100, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",ISOLATE
...,...,...,...,...,...,...,...
775,"[Being, on, the, inclusion, membrane, ,, CT006...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",775,"[[-2.911547183990478, -3.356550693511963, 7.33...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",ISOLATE
776,"[Based, on, the, results, of, AZCL, screening,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",776,"[[-2.804240465164184, -3.308460235595703, 7.10...","[-100, 2, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 10...",ISOLATE
777,"[Meanwhile, ,, different, ICEs, have, been, de...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",777,"[[-2.845585346221924, -3.365410089492798, 7.31...","[-100, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, -100, ...","[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 9, ...",ISOLATE
778,"[This, might, benefit, from, the, symbiosis, r...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",778,"[[-2.865844011306762, -3.423704624176025, 7.28...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 8, 8, 9, ...",ISOLATE


In [6]:
label_ids = df[["label_ids"]]

In [7]:
annotations = []
current_annotation = None

for j in range(len(df)):
	for i in zip(df.loc[j, "tokens"], df.loc[j, "ner_tags"]):
		if i[1] != "O":
			if current_annotation is None:
				current_annotation = { "text": i[0]}
			else:
				current_annotation["text"] += " " + i[0]
		else:
			if current_annotation is not None:
				annotations.append(current_annotation)
				current_annotation = None

if current_annotation is not None:
	annotations.append(current_annotation)


### Predict sentences again

In [8]:
ner = "ISOLATE"

In [9]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=0)


In [10]:
# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:
            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

We also found 20 nonribosomally synthesized peptide clusters and one hybrid polyketide-NRPS cluster in Desulfovibrio piger (Table S1).
[]
The isobutanol biosynthesis pathway in S. blattae (p424IbPSO) uses the endogenous NADPH-dependent alcohol dehydrogenase YqhD to complete the pathway.
[]
Strain ME23 specifically was identified as a representative strain from a 2014 outbreak on potato.
[]
BRCA2 tumours expressed cyclin D1 and D3, cyclin D kinase (CDK) 4 and the CDK inhibitors, p16, p21, and p27, which were all downregulated in BRCA1.
[]
One contained 12 of the 14 milk strains, and the other contained the remaining 2 milk strains (strains R0052 and CAUH18) and all the whisky strains.
[{'entity_group': 'B', 'score': 0.8705721, 'word': 'milk', 'start': 27, 'end': 31}, {'entity_group': 'B', 'score': 0.8759321, 'word': 'milk', 'start': 81, 'end': 85}, {'entity_group': 'B', 'score': 0.45705488, 'word': 'whisky', 'start': 133, 'end': 139}]
The single‐deletion strains H‐ΔcatA and H‐ΔcatA2 acc

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[]
Both D7795 and CP255 lacked ΦSE20 and instead carried P88-like and Fels2-like prophages.
[]
Here, we report the draft genome sequence (4.2 Mb) of Sphingobium quisquiliarum strain P25T, a natural lin (genes involved in degradation of hexachlorocyclohexane [HCH] isomers) variant genotype, isolated from a heavily contaminated (450 mg HCH/g of soil) HCH dumpsite.
[{'entity_group': 'B', 'score': 0.6753174, 'word': 'heavily', 'start': 212, 'end': 219}, {'entity_group': 'I', 'score': 0.7009007, 'word': 'contaminated', 'start': 220, 'end': 232}, {'entity_group': 'I', 'score': 0.6978344, 'word': 'soil ) hch dumpsite', 'start': 250, 'end': 268}]
Interestingly, 2-SC 58 was five to six times less potent at inhibiting the growth of MRC-5 cells (GI_50 = 26.9 ± 3.2 μM).
[]
MAFF 211472 was isolated from ginger in Nakamura, Kochi Prefecture, Japan, in 1997.
[{'entity_group': 'I', 'score': 0.48202893, 'word': 'ginger', 'start': 30, 'end': 36}]
Moreover, due to enormous metabolic flexibility, S. therm

In [11]:
predictions = [p for p in predictions if p]

In [12]:
predictions

[[{'entity_group': 'B',
   'score': 0.8705721,
   'word': 'milk',
   'start': 27,
   'end': 31},
  {'entity_group': 'B',
   'score': 0.8759321,
   'word': 'milk',
   'start': 81,
   'end': 85},
  {'entity_group': 'B',
   'score': 0.45705488,
   'word': 'whisky',
   'start': 133,
   'end': 139}],
 [{'entity_group': 'B',
   'score': 0.78533965,
   'word': 'fresh',
   'start': 128,
   'end': 133},
  {'entity_group': 'I',
   'score': 0.7587848,
   'word': 'stool sample',
   'start': 134,
   'end': 146},
  {'entity_group': 'I',
   'score': 0.6795437,
   'word': 'a 28 - year',
   'start': 162,
   'end': 171},
  {'entity_group': 'I',
   'score': 0.7569978,
   'word': 'old',
   'start': 172,
   'end': 175},
  {'entity_group': 'I',
   'score': 0.7233827,
   'word': 'woman',
   'start': 186,
   'end': 191}],
 [{'entity_group': 'B',
   'score': 0.6753174,
   'word': 'heavily',
   'start': 212,
   'end': 219},
  {'entity_group': 'I',
   'score': 0.7009007,
   'word': 'contaminated',
   'start': 22

In [13]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    pred = pred.replace(" - ","-").replace("- ","-").replace(" -","-")
    for annotation in annotations:
        ann = annotation["text"]
        ann = ann.lower()
        if ann in pred:
            count +=1
    if count ==0:
        print(pred)

fresh
heavily
ginger
nitrate
traditional
human
ice
fermented
human
dead
rectal
bovine
highly alkaline
solar
inner
armadillos
gastrointestinal
cow
soybean
hydrothermal
honey
german
apple
colorectal
root
small
human
materials
gold
environments
low
contaminated
biofilm
agar
shallow
melt
rumen
sediment
oral
peat
resected
welder
mushroom
polluted
acidic
stratified
oxygen rhizome and
field
human
cf patient
nitrocellulose
ice
14-week-old senegalese girl
peanut
soy
symptomatic foliar
hydraulically
feces
hydrocarbon
contaminated
tidal
soft
fecal
human
surface
pig
root
ice
porcine
vacuum
surgically
microbial
diseased
fecal
deep
meat
ginger
freshwater


In [14]:
merged

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[We, also, found, 20, nonribosomally, synthesi...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.956822395324707, -3.364196300506592, 7.22...","[-100, 2, 2, 2, 2, 2, -100, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 4, 4, 4, 5, 6, 7, 8, 9, ...",ISOLATE
1,"[The, isobutanol, biosynthesis, pathway, in, S...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.948723077774048, -3.370908737182617, 7.29...","[-100, 2, 2, -100, -100, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 1, 1, 2, 3, 4, 5, 5, 6, 6, 6, 7, ...",ISOLATE
2,"[Strain, ME23, specifically, was, identified, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",2,"[[-2.8552932739257812, -3.23327374458313, 6.88...","[-100, 2, 2, -100, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11...",ISOLATE
3,"[BRCA2, tumours, expressed, cyclin, D1, and, D...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.735277652740478, -3.456892967224121, 7.22...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",ISOLATE
4,"[One, contained, 12, of, the, 14, milk, strain...","[O, O, O, O, O, O, B, O, O, O, O, O, O, O, O, ...",4,"[[-2.889482736587524, -3.250913143157959, 7.06...","[-100, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",ISOLATE
...,...,...,...,...,...,...,...
775,"[Being, on, the, inclusion, membrane, ,, CT006...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",775,"[[-2.911547183990478, -3.356550693511963, 7.33...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",ISOLATE
776,"[Based, on, the, results, of, AZCL, screening,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",776,"[[-2.804240465164184, -3.308460235595703, 7.10...","[-100, 2, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 10...",ISOLATE
777,"[Meanwhile, ,, different, ICEs, have, been, de...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",777,"[[-2.845585346221924, -3.365410089492798, 7.31...","[-100, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, -100, ...","[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 9, ...",ISOLATE
778,"[This, might, benefit, from, the, symbiosis, r...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",778,"[[-2.865844011306762, -3.423704624176025, 7.28...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 8, 8, 9, ...",ISOLATE


In [15]:
merged

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[We, also, found, 20, nonribosomally, synthesi...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.956822395324707, -3.364196300506592, 7.22...","[-100, 2, 2, 2, 2, 2, -100, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 4, 4, 4, 5, 6, 7, 8, 9, ...",ISOLATE
1,"[The, isobutanol, biosynthesis, pathway, in, S...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.948723077774048, -3.370908737182617, 7.29...","[-100, 2, 2, -100, -100, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 1, 1, 2, 3, 4, 5, 5, 6, 6, 6, 7, ...",ISOLATE
2,"[Strain, ME23, specifically, was, identified, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",2,"[[-2.8552932739257812, -3.23327374458313, 6.88...","[-100, 2, 2, -100, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11...",ISOLATE
3,"[BRCA2, tumours, expressed, cyclin, D1, and, D...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.735277652740478, -3.456892967224121, 7.22...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",ISOLATE
4,"[One, contained, 12, of, the, 14, milk, strain...","[O, O, O, O, O, O, B, O, O, O, O, O, O, O, O, ...",4,"[[-2.889482736587524, -3.250913143157959, 7.06...","[-100, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",ISOLATE
...,...,...,...,...,...,...,...
775,"[Being, on, the, inclusion, membrane, ,, CT006...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",775,"[[-2.911547183990478, -3.356550693511963, 7.33...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",ISOLATE
776,"[Based, on, the, results, of, AZCL, screening,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",776,"[[-2.804240465164184, -3.308460235595703, 7.10...","[-100, 2, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 10...",ISOLATE
777,"[Meanwhile, ,, different, ICEs, have, been, de...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",777,"[[-2.845585346221924, -3.365410089492798, 7.31...","[-100, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, -100, ...","[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 9, ...",ISOLATE
778,"[This, might, benefit, from, the, symbiosis, r...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",778,"[[-2.865844011306762, -3.423704624176025, 7.28...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 8, 8, 9, ...",ISOLATE


In [16]:
## TODO: also split the annotations by "-"
# Create an empty dataframe to store the results
results_df = pd.DataFrame(columns=['NER', 'Prediction'])

ners = ["STRAIN", "METABOLITE", "SPECIES", "ISOLATE", "COMPOUND","MEDIUM", "ORGANISM", "PHENOTYPE", "PROTEIN","MORPHOLOGY","EFFECT","DISEASE"]

for ner in ners:
    test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
    test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
    merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
    merged.loc[:, "ner"] = ner

    annotations = []
    current_annotation = None

    for j in range(len(merged)):
        for i in zip(merged.loc[j, "tokens"], merged.loc[j, "ner_tags"]):
            if i[1] != "O":
                if current_annotation is None:
                    current_annotation = {"text": i[0]}
                else:
                    current_annotation["text"] += " " + i[0]
            else:
                if current_annotation is not None:
                    annotations.append(current_annotation)
                    current_annotation = None

    if current_annotation is not None:
        annotations.append(current_annotation)

    path = f"../NER_output/{ner}"

    tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path)

    nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
                   aggregation_strategy="average", device=0)

    # Specify the path to the JSONL file
    jsonl_file = f"../NER/{ner}/test.jsonls"

    predictions = []
    # Open the JSONL file
    with jsonlines.open(jsonl_file) as reader:
        # Iterate over each line in the file
        for line in reader:
            for l in line:

                # Get the value from the "text" field
                text = l["data"]["text"]
                # Pass the value to the nlp() function
                predict = nlp(text)
                predictions.append(predict)
    predictions = [p for p in predictions if p]

    # Store the predictions in the dataframe
    for prediction in predictions:
        pred = prediction[0]["word"]
        count = 0
        pred = pred.replace(" - ", "-").replace("- ", "-").replace(" -", "-").lower()
        for annotation in annotations:
            ann = annotation["text"]
            ann = ann.lower()
            if ann in pred:
                count += 1
        if count == 0:
            results_df = pd.concat([results_df, pd.DataFrame({'NER': [ner], 'Prediction': [pred]})], ignore_index=True)

    annotations_set = [i["text"].lower().split(" ") for i in annotations]
    annotations_set = set(itertools.chain.from_iterable(annotations_set))

    predictions_set = [i[0]["word"].lower().replace(" - ", "-").replace("- ", "-").replace(" -", "-").split(" ") for i in predictions]
    predictions_set = set(itertools.chain.from_iterable(predictions_set))

    ann_not_pred = annotations_set.difference(predictions_set)
    pred_not_ann = predictions_set.difference(annotations_set)

    # with open(f"ner_annotation/{ner}_ann_not_pred.txt", "w") as f:
    #     for i in ann_not_pred:
    #         f.write(i + "\n")
    with open(f"ner_annotation/{ner}_pred_not_ann.txt", "w") as f:
        for i in pred_not_ann:
            f.write(i + "\n")

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python